In [1]:
using AgentBasedModels
abm = AgentBasedModels

using Random
using Distributions
using CUDA

using Makie

┌ Info: Precompiling AgentBasedModels [388cb286-f2b1-4654-a3bb-2e137a39c658]
└ @ Base loading.jl:1317
  ** incremental compilation may be fatally broken for this module **



# Define the agent

In [37]:
m = @agent(saiz,
    
    #Mechanics
    
    [x,y,z,vx,vy,vz]::Local, #Variables
    [Fix,Fiy,Fiz]::Local,    #Interaction forces
    id::Identity,            #Identity
    [m,r]::Local,            #Mass and radius of the model
    [μ,b]::Global,           #Global parameters of the model
    F₀::Global,         #Matrix of interaction forces
    
    Equation = begin
        d_vx = (-b*vx/m+Fix/m)*dt
        d_vy = (-b*vy/m+Fiy/m)*dt
        d_vz = (-b*vz/m+Fiz/m)*dt
        d_x = vx*dt
        d_y = vy*dt
        d_z = vz*dt
    end,
    
    UpdateInteraction = begin
        dij = sqrt((dx_i-dx_j)^2+(dy_i-dy_j)^2+(dz_i-dz_j)^2)
        rij = r_i+r_j
        if dij < μ*rij && dij > 0
            Fix += F₀*(rij/dij-1)*(μ*rij/dij-1)*(x_i-x_j)/dij
            Fiy += F₀*(rij/dij-1)*(μ*rij/dij-1)*(y_i-y_j)/dij
            Fiz += F₀*(rij/dij-1)*(μ*rij/dij-1)*(z_i-z_j)/dij   
        end
    end,
    
    #Biochemistry
    
    c::Local,             #Biochemical component
    [ci,ni]::Local,       #Interaction parameters for the mean
    [α,n,mm,K]::Global,
    [frange,fmin,fmax]::Global,       #Distance of communication
    [toff,ton]::Global,
    
    Equation = begin
        if t < toff && t > ton
            d_c = (α*(1+c^n)^mm/((1+c^n)^mm+(1+(ci/ni)/K)^(2*mm))-c)*dt
        else
            d_c = 0*dt
        end
    end,
        
    UpdateInteraction= begin
        if dij < frange*rij #Unnecessary to compute dij and rij again, previously computed in UpdateInteraction
            ni += 1
            ci += c_j
        end 
    end,
    
    #Growth
    
    tu::Local,
    [τdiv,σdiv,σc]::Global,

    EventDivision = begin
        if t > tu
            #Choose random direction in unit sphere
            xₐ = Normal(0,1); yₐ = Normal(0,1); zₐ = Normal(0,1)
            Tₐ = sqrt(xₐ^2+yₐ^2+zₐ^2)
            xₐ /= Tₐ;yₐ /= Tₐ;zₐ /= Tₐ    

            #Chose a random distribution
            dist = Uniform(1+σc1,1-σc)

            #Update things of first cell
            x_1 = x+r*xₐ/2; y_1 = y+r*yₐ/2; z_1 = z+r*zₐ/2
            vx_1 = 0.; vy_1 = 0.; vz_1 = 0.
            r_1 = r/2. ^(1. /3)
            m_1 = m/2
            tu_1 = t + Uniform(τdiv-σdiv,τdiv+σdiv)
            c_1 = c*dist

            #Update things of second cell
            x_2 = x+r*xₐ/2; y_2 = y+r*yₐ/2; z_2 = z+r*zₐ/2
            vx_2 = 0.; vy_2 = 0.; vz_2 = 0.
            r_2 = r/2. ^(1. /3)
            m_2 = m/2
            tu_2 = t + Uniform(τdiv-σdiv,τdiv+σdiv)
            c_2 = c*(2-dist)
        end
    end,
)

PARAMETERS
Identity
	 agentId, id,
Global
	 μ, b, F₀, α, n, mm, K, frange, fmin, fmax, toff, ton, τdiv, σdiv, σc,
Local
	 x, y, z, vx, vy, vz, Fix, Fiy, Fiz, m, r, c, ci, ni, tu,


UPDATE RULES
Equation
 begin
    d_vx = ((-b * vx) / m + Fix / m) * dt
    d_vy = ((-b * vy) / m + Fiy / m) * dt
    d_vz = ((-b * vz) / m + Fiz / m) * dt
    d_x = vx * dt
    d_y = vy * dt
    d_z = vz * dt
    if t < toff && t > ton
        d_c = ((α * (1 + c ^ n) ^ mm) / ((1 + c ^ n) ^ mm + (1 + (ci / ni) / K) ^ (2mm)) - c) * dt
    else
        d_c = 0dt
    end
end

EventDivision
 begin
    if t > tu
        xₐ = Normal(0, 1)
        yₐ = Normal(0, 1)
        zₐ = Normal(0, 1)
        Tₐ = sqrt(xₐ ^ 2 + yₐ ^ 2 + zₐ ^ 2)
        xₐ /= Tₐ
        yₐ /= Tₐ
        zₐ /= Tₐ
        dist = Uniform(1 + σc1, 1 - σc)
        x_1 = x + (r * xₐ) / 2
        y_1 = y + (r * yₐ) / 2
        z_1 = z + (r * zₐ) / 2
        vx_1 = 0.0
        vy_1 = 0.0
        vz_1 = 0.0
        r_1 = r / 2.0 ^ (1.0 / 3)
        m_1 

# Compile the model

In [38]:
mCompiled = compile(m,platform="cpu",save="RAM",integrator="Heun",debug=false);

# Initialise the Community

In [39]:
com = Community(mCompiled,N=1)

##########Mechanics##############################
#Initialise global
com.b = 10^-6
com.F₀ = 10^-4 #.*zeros(2,2)
com.μ = 2

#########Division#################################
#Initialise global
com.τdiv = 10
com.σdiv = 0.5
com.σc = 0.01

#########Chemical#################################
#Initialise Global
com.ton = 0 #Start deactivated
com.toff = 100 #Start activated
com.fmin = 0.05
com.fmax = 0.95
com.α = 10
com.n = 2
com.mm = 2
com.K = 0.9
com.frange = 1.2

#########Local parameters and variables###########
#Initialise locals
mᵢ = 10^-6
rᵢ = 5
com.m .= mᵢ
com.r .= rᵢ
com.id .= 1 #Start neutral fate
#Initialise variables
com.x .= 0. #They are only separed in this axis
com.y .= 0.
com.z .= 0.
com.vx .= 0.
com.vy .= 0.
com.vz .= 0.

c₀=3.
com.c .= c₀
com.tu .= 10.

1-element view(::Matrix{Float64}, :, 15) with eltype Float64:
 10.0

In [40]:
println(mCompiled.program)

begin
    #= none:3 =#
    function (com::Community,; dt::Real, tMax::Real, t::Real = com.t, N::Integer = com.N, nMax::Integer = com.N)
        #= none:3 =#
        #= none:6 =#
        dt = Float64(dt)
        #= none:8 =#
        tMax = Float64(tMax)
        #= none:10 =#
        t = Float64(t)
        #= none:12 =#
        N = Int(N)
        #= none:14 =#
        #= none:17 =#
        localV = Array(Float64.([com.local_; Base.zeros(nMax - N, 15)]))
        #= none:19 =#
        localVCopy = zeros(Float64, (size(localV))[1], 10)
        #= none:21 =#
        identityV = Array([Int.(com.identity_); Base.zeros(Int, nMax - N, 2)])
        #= none:23 =#
        globalV = Array(Float64.(com.global_))
        #= none:24 =#
        K₁_ = copy(zeros(Float64, nMax, 7))
        #= none:25 =#
        NV = Threads.Atomic{Int}(N)
        #= none:26 =#
        agentIdMax = Threads.Atomic{Int}(N)
        #= none:27 =#
        commRAM_ = CommunityInTime()
        #= none:30 =#
        #= none:32 =#


                    #= none:258 =#
                    localVCopy[ic1_, 1] = localV[ic1_, 1] + (localVCopy[ic1_, 1] + K₁_[ic1_, 1]) / 2
                    #= none:259 =#
                    localVCopy[ic1_, 2] = localV[ic1_, 2] + (localVCopy[ic1_, 2] + K₁_[ic1_, 2]) / 2
                    #= none:260 =#
                    localVCopy[ic1_, 3] = localV[ic1_, 3] + (localVCopy[ic1_, 3] + K₁_[ic1_, 3]) / 2
                    #= none:261 =#
                    localVCopy[ic1_, 4] = localV[ic1_, 4] + (localVCopy[ic1_, 4] + K₁_[ic1_, 4]) / 2
                    #= none:262 =#
                    localVCopy[ic1_, 5] = localV[ic1_, 5] + (localVCopy[ic1_, 5] + K₁_[ic1_, 5]) / 2
                    #= none:263 =#
                    localVCopy[ic1_, 6] = localV[ic1_, 6] + (localVCopy[ic1_, 6] + K₁_[ic1_, 6]) / 2
                    #= none:264 =#
                    localVCopy[ic1_, 7] = localV[ic1_, 12] + (localVCopy[ic1_, 7] + K₁_[ic1_, 7]) / 2
                end
            #= none:269 =#


                        #= none:411 =#
                        identityV[ic1_, 1] = idNew_
                        #= none:412 =#
                        identityV[ic1New_, 1] = idNew_ + 1
                        #= none:413 =#
                        identityV[ic1New_, 2] = identityV[ic1_, 2]
                    end
                end
            #= none:420 =#
            return nothing
        end
        #= none:424 =#
        #= none:426 =#
        initialiseCopy_!(t, N, dt, localV, localVCopy, identityV, globalV, K₁_, NV)
        #= none:427 =#
        #= none:428 =#
        cleanInteraction_!(t, N, dt, localV, localVCopy, identityV, globalV, K₁_, NV)
        #= none:430 =#
        interactionCompute_!(t, N, dt, localV, localVCopy, identityV, globalV, K₁_, NV)
        #= none:432 =#
        integrationStep_!(t, N, dt, localV, localVCopy, identityV, globalV, K₁_, NV)
        #= none:433 =#
        #= none:435 =#
        ob = Community(t, N, com.declaredSymbols_, (Core.Array(local

In [41]:
c = mCompiled.evolve(com,dt=0.001,tMax=30,nMax=1000);

LoadError: UndefVarError: interactionCompute_! not defined

In [20]:
c[last][:xx];

## Visualization

### Plot community

In [21]:
figure = plotCommunitySpheres(c[50],[:x,:y,:z],:r,:fate)
display(figure)

GLMakie.Screen(...)

### Make video

In [22]:
using Makie
using AbstractPlotting

plotDivisionTree (generic function with 2 methods)

In [44]:
names(Dict([:a=>1]))

LoadError: MethodError: no method matching names(::Dict{Symbol,Int64})
Closest candidates are:
  names(!Matched::DataFrames.Index) at /home/gabriel/.julia/packages/DataFrames/Zx5mm/src/other/index.jl:34
  names(!Matched::Module; all, imported) at reflection.jl:98
  names(!Matched::DataFrames.SubIndex) at /home/gabriel/.julia/packages/DataFrames/Zx5mm/src/other/index.jl:425
  ...

In [41]:
figure = plotDivisionTree(c,:fate)
display(figure)

LoadError: UndefVarError: key not defined